In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tabulate import tabulate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def recipe_recommendation_system():

  # 1차 필터링
  print("만들고 싶은 레시피의 분류를 선택하세요.")
  print("---------------------------------------")
  print("1. 메인반찬")
  print("2. 밑반찬")
  print("3. 국_탕")
  print("4. 찌개")
  print("5. 밥_죽_떡")
  print("6. 면_만두")
  print("7. 김치_젓갈_장류")
  print("8. 양식")
  print("9. 퓨전")
  print("10. 스프")
  print("11. 샐러드")
  print("12. 빵")
  print("13. 디저트")
  print("---------------------------------------")
  dict = {1 : "메인반찬", 2 : "밑반찬", 3 : "국_탕", 4 : "찌개", 5 : "밥_죽_떡", 6 : "면_만두",
        7 : "김치_젓갈_장류", 8 : "양식", 9 : "퓨전", 10 : "스프", 11 : "샐러드", 12 : "빵", 13 : "디저트"}
  n = int(input("번호를 입력하세요 : "))
  ctgy = dict[n]
  print(ctgy + " 선택했습니다.")
  print("---------------------------------------")

  path_ = '/content/drive/Shareddrives/데캡디 TEAM G/DATA/최종 레시피 데이터/'
  path__ = '/content/drive/Shareddrives/데캡디 TEAM G/DATA/대표메뉴 데이터/'

  path_1 = path_ + "최종recipe_" + ctgy + ".csv"
  path_2 = path__ + "대표메뉴_" + ctgy + ".csv"

  data_1 = pd.read_csv(path_1, encoding = 'utf-8')
  data_2 = pd.read_csv(path_2, encoding = 'utf-8')

  idx_1 = data_1[data_1["레시피내용"] == '0'].index
  idx_2 = data_1[data_1["레시피내용"] == '[]'].index
  idx_3 = data_1[data_1["레시피내용"] == 0].index
  data_1 = data_1.drop(idx_1)
  data_1 = data_1.drop(idx_2)
  data_1 = data_1.drop(idx_3)
  data_1 = data_1.drop_duplicates(["레시피내용"], keep = 'first')
  data_1["요리재료_"] = 0
  data_1["요리재료_"] = data_1["요리재료"] + " " + data_1["요리명"]
  new_data = data_1[['레시피 제목', '요리재료', '레시피내용', '요리재료_']]
  new_data.rename(columns = {'레시피 제목' : '대표메뉴명'}, inplace = True)
  data_2["레시피내용"] = 0
  data_2["요리재료_"] = 0
  new_data = pd.concat([data_2, new_data])

  tfidf = TfidfVectorizer(stop_words = 'english')
  tfidf_matrix = tfidf.fit_transform(new_data['요리재료_'].astype('U').values)
  cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
  indices = pd.Series(data_2.index, index = data_2['대표메뉴명']).drop_duplicates()


  # 2차 필터링
  print("선호하는 대표메뉴 3가지를 선택하세요.")
  print("---------------------------------------")
  menu_list = []
  for i in range(10) :
      menu_list.append(data_2["대표메뉴명"][i])
  for i in range(10) :
      print(str(i+1) + ". " + menu_list[i])
  print("---------------------------------------")
  i = 1
  menu = []
  a, b, c = map(int,input("3개의 번호를 입력하세요(ex:1 2 3) : ").split())
  menu.append(menu_list[a-1])
  menu.append(menu_list[b-1])
  menu.append(menu_list[c-1])
  print(menu[0] + ', ' + menu[1] + ', ' + menu[2] + " 선택했습니다.")
  print("---------------------------------------")
  print(menu) #이거 없애도 되지 않낭

  def get_recommendations_(recipe, cosine_sim = cosine_sim):
      # 선택한 레시피로부터 해당되는 인덱스를 받아온다.
      recipe2idx = {}
      for i, c in enumerate(new_data["대표메뉴명"]) :
          recipe2idx[i] = c
      idx2recipe = {}
      for i, c in recipe2idx.items() :
          idx2recipe[c] = i
      
      # 모든 레시피에 대해서 해당 레시피와의 유사도 계산
      idx = idx2recipe[recipe]
      sim_scores = [(i, c) for i, c in enumerate(cosine_sim[idx]) if i != idx]

      # 유사도에 따라 레시피들을 정렬
      sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse=True)

      scores = np.array(sim_scores)
      mean = scores[:, 1].mean()
      std = scores[:, 1].std()
      new_sim_scores = [i for i in sim_scores if i[1] > (mean + std)]

      # 가장 유사한 레시피의 인덱스
      recipe_indices = [i[0] for i in new_sim_scores]

      # 가장 유사한 레시피의 제목.
      return new_data[["대표메뉴명", "요리재료", "레시피내용", "요리재료_"]].iloc[recipe_indices]

  data_filtered = pd.DataFrame()
  for i in range(len(menu)) :
      data_filtered = pd.concat([data_filtered, get_recommendations_(menu[i])])


  # 3차 필터링(재료 입력)
  input_ingre = input("사용할 재료를 입력하세요 : ")
  data_filtered = data_filtered.reset_index()
  length = len(data_filtered)
  data_filtered.loc[length] = [0, "", [], "", input_ingre]

  tfidf = TfidfVectorizer(stop_words = 'english')
  tfidf_matrix = tfidf.fit_transform(data_filtered['요리재료_'])
  cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

  def get_recommendations(cosine_sim = cosine_sim):
      # 모든 재료에 대해서 입력한 재료와의 유사도 계산
      idx = length
      sim_scores = [(i, c) for i, c in enumerate(cosine_sim[idx]) if i != idx]

      # 유사도에 따라 레시피들을 정렬
      sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse=True)

      scores = np.array(sim_scores)
      mean = scores[:, 1].mean()
      std = scores[:, 1].std()
      new_sim_scores = [i for i in sim_scores if i[1] > (mean + std)]

      # 가장 유사한 레시피의 인덱스
      recipe_indices = [i[0] for i in new_sim_scores[0:10]]

      # 가장 유사한 레시피의 10개의 제목
      df = data_filtered[["대표메뉴명", "요리재료", "레시피내용"]]
      df.rename(columns= {'대표메뉴명':'메뉴명'}, inplace = True)
      df = df.iloc[recipe_indices]
      index_list = [1,2,3,4,5,6,7,8,9,10]
      df.set_index(keys = [index_list],inplace = True)
      return df

  recommendation_df = get_recommendations()
  print()
  print("---------------------------------------")
  print("레시피 추천 목록")
  print("---------------------------------------")
  print(tabulate(recommendation_df, headers='keys', tablefmt='psql', showindex=True))

  # recipe_num = int(input("확인하고 싶은 레시피 번호를 입력하세요 : "))
  # recipe_list = recommendation_df["레시피내용"].iloc[recipe_num-1].split(',')
  # print(*recipe_list)

In [ ]:
recipe_recommendation_system()

만들고 싶은 레시피의 분류를 선택하세요.
---------------------------------------
1. 메인반찬
2. 밑반찬
3. 국_탕
4. 찌개
5. 밥_죽_떡
6. 면_만두
7. 김치_젓갈_장류
8. 양식
9. 퓨전
10. 스프
11. 샐러드
12. 빵
13. 디저트
---------------------------------------
번호를 입력하세요 : 3
국_탕 선택했습니다.
---------------------------------------
선호하는 대표메뉴 3가지를 선택하세요.
---------------------------------------
1. 미역국
2. 닭볶음탕
3. 콩나물국
4. 된장국
5. 어묵탕
6. 떡국
7. 계란국
8. 육개장 
9. 삼계탕
10. 북어국
---------------------------------------
3개의 번호를 입력하세요(ex:1 2 3) : 1 2 3
미역국, 닭볶음탕, 콩나물국 선택했습니다.
---------------------------------------
['미역국', '닭볶음탕', '콩나물국']
사용할 재료를 입력하세요 : 닭 당근 양파 감자

---------------------------------------
레시피 추천 목록
---------------------------------------
+----+-------------------------------------------------------------------------+-----------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------